In [9]:
using DataFrames
using CSV
using DataFramesMeta
using BioStructures
using Bio3DView
trueData = "/home/hacquard/fullseqdesign_rosetta/pdb.full"
resData = "/home/hacquard/fullseqdesign_rosetta/allpositions/rosetta"
header = [:atom, :id, :name, :radical, :chain, :radical_id, :x, :y, :z, :occupancy, :bfactor, :element]
types = convert(Vector{Type},[String7, Int64, String3, String3, String1, Int64, Float64, Float64, Float64, Float64, Float64, String1])
function loadpdb(file::String)::DataFrame
    filter(:atom => ==("ATOM"),dropmissing( CSV.read(file, DataFrame; delim=" ", ignorerepeated=true, select=header, header=header, silencewarnings=true, types=types)[:, header]))
end
function sequence(protein::DataFrame)::DataFrame
    unique(select(groupby(protein, :radical_id), :radical))
end
function compare(a::DataFrame,b::DataFrame)::DataFrame
    comparaison = innerjoin(a,b;  on=:radical_id, makeunique=true)
    @transform(
    comparaison,
    :correct = (:radical .== :radical_1)
)
end

compare (generic function with 1 method)

In [146]:
comparaison[2,:radical] == comparaison[2,:radical_1]

true

In [9]:
comparaison = innerjoin((sequence ∘ loadpdb).(["$trueData/1ABO.pdb", "$resData/1ABO.rosetta_0001.pdb"])...; on=:radical_id, makeunique=true)
comparaison = transform(
    comparaison,
    [:radical,:radical_1]=> 
    ((x,y) -> x.==y) => 
    :correct
)
count(comparaison.correct)

22

In [10]:
comparaison

Row,radical_id,radical,radical_1,correct
,Int64,String3,String3,Bool
1,64,ASN,PRO,false
2,65,LEU,LEU,true
3,66,PHE,TYR,false
4,67,VAL,VAL,true
5,68,ALA,ALA,true
6,69,LEU,LEU,true
7,70,TYR,THR,false
8,71,ASP,ASP,true
9,72,PHE,LEU,false


talaris2013|14 et score12